# LocalAERProvider and LocalAERBackend Demo

This notebook showcases how to use the `LocalAERProvider` and `LocalAERBackend` classes, which enable quantum circuit execution using a local Qiskit Aer simulator, integrated with qBraid-compatible tooling.

In this notebook, we will:

1. Construct a basic quantum circuit.
2. Use `LocalAERProvider` to:
   - List available local simulation backends.
   - Retrieve a specific backend.
3. Use `LocalAERBackend` to:
   - Convert the quantum circuit into an executable format.
   - Submit and execute the circuit locally.

> 💡 Before proceeding, make sure the required dependencies are installed: `qiskit`, `qbraid`, and other supporting libraries!

## 1. Setup and Imports

We begin by importing the necessary libraries, including the local AER provider and backend classes.

In [1]:
# Import Qiskit components for circuit creation
from qiskit import QuantumCircuit

# Import the LocalAERProvider for local execution
from quantum_executor.local_aer import LocalAERProvider

## 2. Creating a Simple Quantum Circuit

We create a basic quantum circuit that will be used to test the backend functionalities.
Here, we create a Bell state circuit.

In [2]:
# Create a simple 2-qubit quantum circuit that creates a Bell state
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()
qc.draw()

┌───┐      ░ ┌─┐   
   q_0: ┤ H ├──■───░─┤M├───
        └───┘┌─┴─┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├─░──╫─┤M├
             └───┘ ░  ║ └╥┘
meas: 2/══════════════╩══╩═
                      0  1

## 3. Testing the LocalAERProvider

The `LocalAERProvider` class provides methods to list available quantum devices and to retrieve a specific device by its identifier.

In [3]:
# Create a LocalAERProvider instance
provider = LocalAERProvider()

# List available devices
devices = provider.get_devices()
print("List of available devices:")
for dev in devices:
    print(" -", dev)

List of available devices:
 - LocalAERBackend('aer_simulator')
 - LocalAERBackend('fake_algiers')
 - LocalAERBackend('fake_almaden')
 - LocalAERBackend('fake_armonk')
 - LocalAERBackend('fake_athens')
 - LocalAERBackend('fake_auckland')
 - LocalAERBackend('fake_belem')
 - LocalAERBackend('fake_boeblingen')
 - LocalAERBackend('fake_bogota')
 - LocalAERBackend('fake_brisbane')
 - LocalAERBackend('fake_brooklyn')
 - LocalAERBackend('fake_burlington')
 - LocalAERBackend('fake_cairo')
 - LocalAERBackend('fake_cambridge')
 - LocalAERBackend('fake_casablanca')
 - LocalAERBackend('fake_cusco')
 - LocalAERBackend('fake_essex')
 - LocalAERBackend('fake_fez')
 - LocalAERBackend('fake_fractional')
 - LocalAERBackend('fake_geneva')
 - LocalAERBackend('fake_guadalupe')
 - LocalAERBackend('fake_hanoi')
 - LocalAERBackend('fake_jakarta')
 - LocalAERBackend('fake_johannesburg')
 - LocalAERBackend('fake_kawasaki')
 - LocalAERBackend('fake_kolkata')
 - LocalAERBackend('fake_kyiv')
 - LocalAERBackend('fak

Local Backends are always online.

In [4]:
# Retrieve a noiseless device
# default noiseless simulator: device_id="aer_simulator"
device_id = "aer_simulator"
noiseless_simulator = provider.get_device(device_id)
print(f"\nRetrieved device using device_id '{device_id}':")
print(noiseless_simulator)
print("Status:", noiseless_simulator.status())


Retrieved device using device_id 'aer_simulator':
LocalAERBackend('aer_simulator')
Status: DeviceStatus.ONLINE


In [5]:
# Retrieve a noisy device
# (LocalAERrovider support noisy simulators through qiskit_ibm_runtime FakeProviderForBackendV2)
device_id = "fake_torino"
noisy_simulator = provider.get_device(device_id)
print(f"\nRetrieved device using device_id '{device_id}':")
print(noisy_simulator)
print("Status:", noisy_simulator.status())


Retrieved device using device_id 'fake_torino':
LocalAERBackend('fake_torino')
Status: DeviceStatus.ONLINE


## 4. Testing the LocalAERBackend

Now that we have retrieved a noiseless and noisy simulators, we test:
- The **transform** method to transpile the circuit.
- The **submit** method to execute the circuit on the local simulators.

The submit method requires the `shots` parameter (and optionally a random seed).

### 4.1 Testing the `transform` Method

The `transform` method transpiles the input quantum circuit so that it can be executed on the specified device.

The `aer_simulator`, being a perfect simulator, does not require any transformation of the input circuit.

In [6]:
aer_simulator_transformed_qc = noiseless_simulator.transform(qc)

aer_simulator_transformed_qc.draw()

┌───┐      ░ ┌─┐   
   q_0: ┤ H ├──■───░─┤M├───
        └───┘┌─┴─┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├─░──╫─┤M├
             └───┘ ░  ║ └╥┘
meas: 2/══════════════╩══╩═
                      0  1

On the other hand, the noisy simulators mimic the behaviour and topology of a real quantum computer, thus requiring a transpilation of the circuit.

In [7]:
fake_torino_transformed_qc = noisy_simulator.transform(qc)

fake_torino_transformed_qc.draw()

global phase: 3π/4
          ┌─────────┐┌────┐ ┌───────┐    ┌────┐┌─────────┐ ░    ┌─┐
q_1 -> 65 ┤ Rz(π/2) ├┤ √X ├─┤ Rz(π) ├──■─┤ √X ├┤ Rz(π/2) ├─░────┤M├
          ├─────────┤├────┤┌┴───────┴┐ │ └────┘└─────────┘ ░ ┌─┐└╥┘
q_0 -> 66 ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─■───────────────────░─┤M├─╫─
          └─────────┘└────┘└─────────┘                     ░ └╥┘ ║ 
  meas: 2/════════════════════════════════════════════════════╩══╩═
                                                              0  1

### 4 Testing the `submit` Method

We now submit the quantum circuit for simulation using the `submit` method.
We provide the required `shots` parameter and an optional seed.
The returned job is a `qbraid.runtime.ibm.QiskitJob` object.

In [8]:
job = noiseless_simulator.submit(aer_simulator_transformed_qc, shots=1024)

# Wait for the job to finish
job.wait_for_final_state()

print("Metadata:", job.metadata())

# Print the job result object
print(job.result())

# Print the counts of the measurement results
print(job.result().data.get_counts())

Metadata: {'job_id': '203911d6-902c-441d-b4e7-f73799217fbc', 'status': <COMPLETED: 'job has successfully run'>}
Result(
  device_id=aer_simulator,
  job_id=203911d6-902c-441d-b4e7-f73799217fbc,
  success=True,
  data=GateModelResultData(measurement_counts={'00': 536, '11': 488}, measurements=array(shape=(1024, 1), dtype=uint8)),
  version=2
)
{'00': 536, '11': 488}


Without a seed every execution is different:

In [9]:
job1 = noiseless_simulator.submit(aer_simulator_transformed_qc, shots=1024)
job1.wait_for_final_state()
print(job1.result().data.get_counts())

{'00': 517, '11': 507}


With a seed, instead, we can get the same result every time we run the code:

In [10]:
job2 = noiseless_simulator.submit(aer_simulator_transformed_qc, shots=1024, seed=42)
job2.wait_for_final_state()
print(job2.result().data.get_counts())

job3 = noiseless_simulator.submit(aer_simulator_transformed_qc, shots=1024, seed=42)
job3.wait_for_final_state()
print(job3.result().data.get_counts())

{'00': 521, '11': 503}
{'00': 521, '11': 503}


The same applies for the noisy simulator, the seed controls also the noise. Without a seed:

In [11]:
job4 = noisy_simulator.submit(fake_torino_transformed_qc, shots=1024)
job4.wait_for_final_state()
print(job4.result().data.get_counts())

job5 = noisy_simulator.submit(fake_torino_transformed_qc, shots=1024)
job5.wait_for_final_state()
print(job5.result().data.get_counts())

{'00': 496, '01': 18, '10': 10, '11': 500}
{'00': 483, '01': 28, '10': 11, '11': 502}


With a seed:

In [12]:
job6 = noisy_simulator.submit(fake_torino_transformed_qc, shots=1024, seed=42)
job6.wait_for_final_state()
print(job6.result().data.get_counts())

job7 = noisy_simulator.submit(fake_torino_transformed_qc, shots=1024, seed=42)
job7.wait_for_final_state()
print(job7.result().data.get_counts())

{'00': 517, '01': 7, '10': 16, '11': 484}
{'00': 517, '01': 7, '10': 16, '11': 484}


If we don't transpile the circuit first, the execution may fail:

In [13]:
noisy_simulator.submit(qc, shots=1024)

IBMInputValueError: 'The instruction h on qubits (0,) is not supported by the target system. Circuits that do not match the target hardware definition are no longer supported after March 4, 2024. See the transpilation documentation (https://docs.quantum.ibm.com/guides/transpile) for instructions to transform circuits and the primitive examples (https://docs.quantum.ibm.com/guides/primitives-examples) to see this coupled with operator transformations.'